In [26]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

from ecmwfapi import ECMWFDataServer
import os
import sys
import dask
import numpy as np
from esio import download
import datetime
from esio import EsioData as ed
import pandas as pd
from datetime import timedelta

import calendar
import datetime

import os
from ecmwfapi import ECMWFDataServer

'''
Download models with sea ice forecasts within the 2s2 forecast data archive.

https://software.ecmwf.int/wiki/display/S2S/Models

'''

'\nDownload models with sea ice forecasts within the 2s2 forecast data archive.\n\nhttps://software.ecmwf.int/wiki/display/S2S/Models\n\n'

In [27]:
# meteofr always uses same version which is 2014-12-01
# from 1993 to one year ago

# the MVDs are on 1, 8, 15, 22 
dataclass='s2s'
run_type='reforecast'
cmod = 'cma'
versions = datetime.datetime(2014, 5, 1) # 

cmod = 'ncep' # has same format as cma but different version
versions = datetime.datetime(2011, 3, 1) # 

vdate = versions.strftime('%Y-%m-%d')
print('The version date for model ', cmod, ' is ',vdate)

The version date for model  ncep  is  2011-03-01


In [28]:
E = ed.EsioData.load()
main_dir = E.model_dir

In [29]:
# a starting point
mod_dicts = {}

mod_dicts['cma'] = {
 "class": "s2",
 "dataset": "s2s",
 "date": "2014-05-01",                                                               
 "hdate": "2005-07-11",                                                          
 "expver": "prod",
 "levtype": "sfc",
 "model": "glob",
 "origin": "babj",
 "param": "31",
 "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
 "stream": "enfh",
 "number": "1/2/3",
 "time": "00:00:00",
 "type": "pf",
 "target": "output",
 }

mod_dicts['ncep'] = {
 "class": "s2",
 "dataset": "s2s",
 "date": "2014-05-01",                                                               
 "hdate": "2005-07-11",                                                          
 "expver": "prod",
 "levtype": "sfc",
 "model": "glob",
 "origin": "kwbc",
 "param": "31",
 "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
 "stream": "enfh",
 "number": "1/2/3",
 "time": "00:00:00",
 "type": "pf",
 "target": "output",
 }


In [30]:
mod_dicts

{'cma': {'class': 's2',
  'dataset': 's2s',
  'date': '2014-05-01',
  'expver': 'prod',
  'hdate': '2005-07-11',
  'levtype': 'sfc',
  'model': 'glob',
  'number': '1/2/3',
  'origin': 'babj',
  'param': '31',
  'step': '0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440',
  'stream': 'enfh',
  'target': 'output',
  'time': '00:00:00',
  'type': 'pf'},
 'ncep': {'class': 's2',
  'dataset': 's2s',
  'date': '2014-05-01',
  'expver': 'prod',
  'hdate': '2005-07-11',
  'levtype': 'sfc',
  'model': 'glob',
  'number': '1/2/3',
  'origin': 'kw

In [31]:
@dask.delayed

def download_month(config_dict):
    # Start server
    cserver = ECMWFDataServer()
    cserver.retrieve(config_dict)
    return 1

In [32]:
# just for testing
cy=2000
month=1

DS = datetime.datetime(cy, month, 1) 
DE = datetime.datetime(cy, month+1, 1)
alldates = pd.date_range(start=DS, end=DE,freq='D')
alldates = pd.DatetimeIndex(alldates)
hdates = [x.strftime('%Y-%m-%d/') for x in alldates]
hdates="".join(hdates)
hdates=hdates[0:(len(hdates)-1)] # remove last slash
print(hdates)
print(hdates[0:4]+'_'+hdates[5:7])

2000-01-01/2000-01-02/2000-01-03/2000-01-04/2000-01-05/2000-01-06/2000-01-07/2000-01-08/2000-01-09/2000-01-10/2000-01-11/2000-01-12/2000-01-13/2000-01-14/2000-01-15/2000-01-16/2000-01-17/2000-01-18/2000-01-19/2000-01-20/2000-01-21/2000-01-22/2000-01-23/2000-01-24/2000-01-25/2000-01-26/2000-01-27/2000-01-28/2000-01-29/2000-01-30/2000-01-31/2000-02-01
2000_01


In [ ]:
# super fast! cannot do this for ecmwf and ukmo give their method of
# changing version number every date
for cy in np.arange(1999,2011,1):
    X = 1
    for month in np.arange(1,13,1):
        DS = datetime.datetime(cy, month, 1) 
        if month<12:
            DE = datetime.datetime(cy, month+1, 1)
        else:
            DE = datetime.datetime(cy+1, 1, 1)
        alldates = pd.date_range(start=DS, end=DE,freq='D')
        alldates = pd.DatetimeIndex(alldates)
        hdates = [x.strftime('%Y-%m-%d/') for x in alldates]
        hdates="".join(hdates)
        hdates=hdates[0:(len(hdates)-1)] # remove last slash

        cdict = mod_dicts[cmod]
        cdict['date'] = vdate
        cdict['hdate'] = hdates
        target = os.path.join(main_dir, cmod, run_type, 'native',
                                  cmod+'_'+hdates[0:4]+'_'+hdates[5:7]+'.grib')
        cdict['target'] = target
        cdict['expect'] = 'any'
        print('output file is ',target)
        X = X + download_month(cdict)

    X.compute()

output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_01.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_02.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_03.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_04.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_05.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_06.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_07.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_08.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_09.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_10.grib
output file is  /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_1999_11.grib

Error contacting the WebAPI, retrying in 60 seconds ...
2019-07-21 11:47:15 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https://apps.ecmwf.int/webapi-activity/
2019-07-21 11:47:15 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https://apps.ecmwf.int/webapi-activity/
2019-07-21 11:47:15 Request submitted
2019-07-21 11:47:15 Request id: 5d34b333c8ca19f3355c9b2a
2019-07-21 11:47:15 Request is submitted
2019-07-21 11:47:15 Request submitted
2019-07-21 11:47:15 Request id: 5d34b333cad519fa2050bf51
2019-07-21 11:47:15 Request is submitted
2019-07-21 11:47:15 Request submitted
2019-07-21 11:47:15 Request id: 5d34b333c7db1c98de507dfe
2019-07-21 11:47:15 Request is submitted
2019-07-21 11:47:15 Request submit

Calling 'nice mars /tmp/cb/71/tmp-_marsZdzYWV.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190721.184723 - Welcome to MARS
mars - INFO   - 20190721.184723 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190721.184723 - MARS Client version: 6.25.3
mars - INFO   - 20190721.184723 - MIR version: 1.2.5
mars - INFO   - 20190721.184723 - Using ecCodes version 2.12.6
mars - INFO   - 20190721.184723 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190721.184723 - Maximum retrieval size is 30.00 G
retrieve,origin=kwbc,hdate=1995-02-01/1995-02-02/1995-02-03/1995-02-04/1995-02-05/1995-02-06/1995-02-07/1995-02-08/1995-02-09/1995-02-10/1995-02-11/1995-02-12/1995-02-13/1995-02-14/1995-02-15/1995-02-16/19

Calling 'nice mars /tmp/22/17/tmp-_marsLygVqo.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190721.184724 - Welcome to MARS
mars - INFO   - 20190721.184724 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190721.184724 - MARS Client version: 6.25.3
mars - INFO   - 20190721.184724 - MIR version: 1.2.5
mars - INFO   - 20190721.184724 - Using ecCodes version 2.12.6
mars - INFO   - 20190721.184724 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190721.184724 - Maximum retrieval size is 30.00 G
retrieve,origin=kwbc,hdate=1995-10-01/1995-10-02/1995-10-03/1995-10-04/1995-10-05/1995-10-06/1995-10-07/1995-10-08/1995-10-09/1995-10-10/1995-10-11/1995-10-12/1995-10-13/1995-10-14/1995-10-15/1995-10-16/19

2019-07-21 11:49:20 Transfer rate 0 byte/s
2019-07-21 11:49:20 ECMWF API python library 1.5.4
2019-07-21 11:49:20 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 11:49:20 ECMWF API python library 1.5.4
2019-07-21 11:49:20 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 11:49:20 ECMWF API python library 1.5.4
2019-07-21 11:49:20 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 11:49:20 ECMWF API python library 1.5.4
2019-07-21 11:49:20 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 11:49:20 ECMWF API python library 1.5.4
2019-07-21 11:49:20 ECMWF API at https://api.ecmwf.int/v1
2019-07-21 11:49:20 ECMWF API python library 1.5.4
2019-07-21 11:49:20 ECMWF API at https://api.ecmwf.int/v1
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the We

...

Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
2019-07-21 13:25:57 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https://apps.ecmwf.int/webapi-activity/
2019-07-21 13:25:57 Request submitted
2019-07-21 13:25:57 Request id: 5d34ca55c7db1c98db507e10
2019-07-21 13:25:57 Request is submitt

2019-07-21 13:27:00 Request is queued
2019-07-21 13:27:00 Request is queued
Calling 'nice mars /tmp/6f/49/tmp-_marsC4DSvs.req'
mars - WARN -2019-07-21 13:27:00 Request is queued

mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190721.202656 - Welcome to MARS
mars - INFO   - 20190721.202656 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190721.202656 - MARS Client version: 6.25.3
mars - INFO   - 20190721.202656 - MIR version: 1.2.5
mars - INFO   - 20190721.202656 - Using ecCodes version 2.12.6
mars - INFO   - 20190721.202656 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190721.202656 - Maximum retrieval size is 30.00 G
retrieve,origin=kwbc,hdate=2010-06-01/2010-06-02/2010-06-03/2010-06-04/2010-06-05/2010-06-0

mars - INFO   - 20190721.202750 - 4128 fields retrieved from 'marsth'
mars - WARN   - 20190721.202750 - Visiting database marsth : expected 0, got 4128
mars - INFO   - 20190721.202750 - Request time:  wall: 54 sec  cpu: 14 sec
mars - INFO   - 20190721.202750 -   Read from network: 111.87 Mbyte(s) in 38 sec  [2.91 Mbyte/sec]
mars - INFO   - 20190721.202750 -   Visiting marsth: wall: 54 sec
mars - INFO   - 20190721.202750 -   Post-processing: wall: 9 sec cpu: 9 sec
mars - INFO   - 20190721.202750 -   Writing to target file: 111.87 Mbyte(s) in < 1 sec [451.60 Mbyte/sec]
mars - INFO   - 20190721.202750 - Memory used: 35.61 Mbyte(s)
mars - INFO   - 20190721.202750 - No errors reported
Process '['nice', 'mars', '/tmp/c0/06/tmp-_marsWHwzcn.req']' finished
2019-07-21 13:27:53 Request is complete
2019-07-21 13:27:53 Transfering 111.871 Mbytes into /home/disk/sipn/nicway/data/model/ncep/reforecast/native/ncep_2010_07.grib
2019-07-21 13:27:53 From https://stream.ecmwf.int/data/atls12/data/data05/

Calling 'nice mars /tmp/4c/59/tmp-_marsDzd7UH.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190721.202846 - Welcome to MARS
mars - INFO   - 20190721.202846 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190721.202846 - MARS Client version: 6.25.3
mars - INFO   - 20190721.202846 - MIR version: 1.2.5
mars - INFO   - 20190721.202846 - Using ecCodes version 2.12.6
mars - INFO   - 20190721.202846 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190721.202846 - Maximum retrieval size is 30.00 G
retrieve,origin=kwbc,hdate=2010-03-01/2010-03-02/2010-03-03/2010-03-04/2010-03-05/2010-03-06/2010-03-07/2010-03-08/2010-03-09/2010-03-10/2010-03-11/2010-03-12/2010-03-13/2010-03-14/2010-03-15/2010-03-16/20

2019-07-21 13:29:27 Transfer rate 3.02196 Mbytes/s
2019-07-21 13:29:28 Transfer rate 3.05027 Mbytes/s
2019-07-21 13:29:33 Transfer rate 3.12051 Mbytes/s
mars - INFO   - 20190721.202941 - 3999 fields retrieved from 'marsth'
mars - WARN   - 20190721.202941 - Visiting database marsth : expected 0, got 3999
mars - INFO   - 20190721.202941 - Request time:  wall: 52 sec  cpu: 15 sec
mars - INFO   - 20190721.202941 -   Read from network: 108.38 Mbyte(s) in 37 sec  [2.95 Mbyte/sec]
mars - INFO   - 20190721.202941 -   Visiting marsth: wall: 52 sec
mars - INFO   - 20190721.202941 -   Post-processing: wall: 9 sec cpu: 9 sec
mars - INFO   - 20190721.202941 -   Writing to target file: 108.38 Mbyte(s) in < 1 sec [459.29 Mbyte/sec]
mars - INFO   - 20190721.202939 - 4128 fields retrieved from 'marsth'
mars - WARN   - 20190721.202939 - Visiting database marsth : expected 0, got 4128
mars - INFO   - 20190721.202939 - Request time:  wall: 53 sec  cpu: 16 sec
mars - INFO   - 20190721.202939 -   Read from 